In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

# Preparing Data

In [2]:
SAMPLE_SIZE = 50
NUM_PREDICTORS = 20
NOISE_VAR = 16

X = np.random.rand(SAMPLE_SIZE, NUM_PREDICTORS)
X_scaled = scale(X, axis=0)
X_norm = normalize(X, norm='l2', axis=0)

In [3]:
y = (10 * X[:, 0] + 5 * X[:, 1] + 0.1 * X[:, 2] + 
     np.random.normal(loc=0, scale=NOISE_VAR, size=1)).reshape(SAMPLE_SIZE,1)
y_scaled = scale(y, axis=0)
y_norm = normalize(y, norm='l2', axis=0)

# Least Squares Linear Regression

In [4]:
clf = LinearRegression(normalize=True)
clf.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [5]:
clf.coef_

array([[  1.00000000e+01,   5.00000000e+00,   1.00000000e-01,
          4.42340545e-15,   2.21859907e-15,  -2.03732261e-15,
          7.16308038e-16,   1.85979654e-15,  -1.70935480e-15,
         -7.75076042e-16,   3.25909205e-15,  -1.86164149e-15,
          2.83106617e-15,   1.33201933e-15,  -2.05567253e-15,
          4.03520185e-15,   5.95150385e-16,  -7.43505596e-16,
         -7.44394084e-15,   1.69944529e-15]])

# Lasso

In [6]:
def lasso(alpha, X, y, normalize=True):
    clf_lasso = Lasso(alpha=alpha, normalize=normalize)
    clf_lasso.fit(X, y)
    return clf_lasso.coef_

In [7]:
alphas = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

d = []
for alpha in alphas:
    coeffs = lasso(alpha, X, y, normalize=False)
    d.append({'alpha': alpha, 
              'beta1': coeffs[0], 
              'beta2': coeffs[1],
              'beta3': coeffs[2]})

df_lasso = pd.DataFrame(d, columns=['alpha', 'beta1', 'beta2', 'beta3'])

In [8]:
df_lasso

,alpha,beta1,beta2,beta3
0,0.001,9.985918,4.988087,0.089558
1,0.010,9.859308,4.880433,0.000000
2,0.050,9.270913,4.352178,0.000000
3,0.100,8.535424,3.691860,0.000000
4,0.150,7.799933,3.031542,0.000000
5,0.200,7.064442,2.371224,0.000000
6,0.250,6.328925,1.710901,0.000000
7,0.300,5.593444,1.050584,0.000000


# Effect of Scaling on Coefficients

In [9]:
# First multiply the first column of X by 10
X_prime = X.copy()
X_prime[:, 0] *= 1000.0

coef = lasso(alpha=0.001, X=X_prime, y=y, normalize=False)
print(coef)

[ 0.00999834  4.99056553  0.08873043  0.         -0.         -0.          0.
  0.         -0.         -0.          0.          0.          0.          0.
  0.         -0.          0.          0.         -0.          0.        ]


In [10]:
# Second divide the first column of X by 10
X_prime = X.copy()
X_prime[:, 0] /= 1000.0

coef = lasso(alpha=0.001, X=X_prime, y=y, normalize=True)
print(coef)

[  9.97067509e+03   4.97388236e+00   7.96037207e-02  -0.00000000e+00
   0.00000000e+00  -0.00000000e+00   0.00000000e+00   0.00000000e+00
  -0.00000000e+00  -0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00  -0.00000000e+00   0.00000000e+00  -0.00000000e+00
   0.00000000e+00   0.00000000e+00  -0.00000000e+00  -0.00000000e+00]


# Ridge

In [11]:
def ridge(alpha, X, y, normalize=True):
    clf_ridge = Ridge(alpha, normalize=normalize)
    clf_ridge.fit(X, y)
    return clf_ridge.coef_    

In [12]:
alphas = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

d = []
for alpha in alphas:
    coeffs = ridge(alpha, X, y, normalize=True)
    d.append({'alpha': alpha, 
              'beta1': coeffs[0][0], 
              'beta2': coeffs[0][1],
              'beta3': coeffs[0][2]})

df_ridge = pd.DataFrame(d, columns=['alpha', 'beta1', 'beta2', 'beta3'])

In [13]:
df_ridge

,alpha,beta1,beta2,beta3
0,0.001,9.978663,4.990029,0.100191
1,0.010,9.792871,4.902452,0.102082
2,0.050,9.080292,4.554690,0.113010
3,0.100,8.377144,4.194972,0.128938
4,0.150,7.812616,3.896186,0.144655
5,0.200,7.343848,3.642635,0.159049
6,0.250,6.944920,3.423832,0.171794
7,0.300,6.599038,3.232465,0.182879


# Effect of Scaling on Coeffs: Ridge

In [14]:
# First multiply the first column of X by 10
X_prime = X.copy()
X_prime[:, 0] *= 1000.0

coef = ridge(alpha=0.1, X=X_prime, y=y, normalize=False)
print(coef)

[[  9.96559000e-03   4.86036655e+00   1.11842971e-01   3.32254763e-02
   -2.68443610e-02  -1.54997019e-02   1.52031409e-02   3.08005697e-02
   -8.52807569e-03   1.45396409e-02   3.16990407e-02   4.53686243e-03
    1.96584559e-03   1.90182252e-02  -7.50298214e-04   5.54301681e-04
   -2.60611567e-02   5.48039359e-02  -8.97384952e-03   4.17872419e-02]]


In [15]:
# Next divide the first column of X by 10
X_prime = X.copy()
X_prime[:, 0] /= 1000.0

coef = ridge(alpha=0.1, X=X_prime, y=y, normalize=False)
print(coef)

[[ 0.22111139  3.34271627 -0.08906815 -1.66148851  4.63391815 -2.19487158
   1.06077118  2.84743225 -0.14246962 -1.63373169  3.44601573  2.09349612
  -0.94504078 -1.78248421  1.240716   -0.96688591 -0.25974884  0.49257941
  -0.55173352 -1.95840359]]
